# Package loading and basic configurations

In [ ]:
%load_ext autoreload
%autoreload 2

# load dependencies'
import matplotlib.pyplot as plt
import scipy
from scipy.misc import derivative
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
from envirocar import TrackAPI, DownloadClient, BboxSelector, TimeSelector, ECConfig

# create an initial but optional config and an api client
config = ECConfig()
track_api = TrackAPI(api_client=DownloadClient(config=config))

# Querying enviroCar Tracks

The following cell queries tracks from the enviroCar API. It defines a bbox for the area of Münster (Germany) and requests 50 tracks in the time interval. 

In [23]:
bbox = BboxSelector([
    7.603312, # min_x
    51.952343, # min_y
    7.65083, # max_x 
    51.974045,  # max_y 
])


# issue a query
track_df = track_api.get_tracks(bbox=bbox,  num_results=50) # requesting 50 tracks inside the bbox

In [24]:
one_track_id = track_df['track.id'].unique()[4]
one_track = track_df[track_df['track.id'] == one_track_id]

## Add elevation

In [25]:
import requests as req
url = 'https://api.opentopodata.org/v1/eudem25m?locations='
def get_elvdata(lat,lng):
    access = url+str(lat)+(',')+str(lng)
    elevation = req.request('GET',access)
    data = eval(elevation.text)
    print(data) # debug code for everyone to check the request status
    return data['results'][0]['elevation']
for i in one_track.index:     
    one_track.loc[i,'elevation'] = get_elvdata(one_track.loc[i,'geometry'].y,one_track.loc[i,'geometry'].x)
#The request takes time
one_track

{'results': [{'elevation': 55.76268768310547, 'location': {'lat': 51.93495048766459, 'lng': 7.651782336187098}}], 'status': 'OK'}


c:\users\pablo\anaconda3\envs\energydem\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\users\pablo\anaconda3\envs\energydem\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


{'results': [{'elevation': 55.712318420410156, 'location': {'lat': 51.93497110400973, 'lng': 7.651832374399817}}], 'status': 'OK'}
{'results': [{'elevation': 55.80480194091797, 'location': {'lat': 51.93479948089254, 'lng': 7.651847605902519}}], 'status': 'OK'}
{'results': [{'elevation': 55.73940658569336, 'location': {'lat': 51.934749342797225, 'lng': 7.652361552459999}}], 'status': 'OK'}
{'results': [{'elevation': 55.97761535644531, 'location': {'lat': 51.934441969067734, 'lng': 7.65273624375139}}], 'status': 'OK'}
{'results': [{'elevation': 56.06355667114258, 'location': {'lat': 51.934276865836864, 'lng': 7.65286357535016}}], 'status': 'OK'}
{'results': [{'elevation': 55.84510803222656, 'location': {'lat': 51.93436620346489, 'lng': 7.6530891342549445}}], 'status': 'OK'}
{'results': [{'elevation': 55.61606979370117, 'location': {'lat': 51.93470892548684, 'lng': 7.6536264596716075}}], 'status': 'OK'}
{'results': [{'elevation': 55.9713134765625, 'location': {'lat': 51.93515214143271, 'l

KeyError: 'results'

## Calculate gradient(Elevation) & add time interval

In [26]:
def distance(lon1,lon2,lat1,lat2):
    R = 6370000 #radius
    φ1=lat1 * np.pi /180
    φ2 = lat2 * np.pi /180
    Δφ = (lat2-lat1) * np.pi /180
    Δλ = (lon2-lon1) * np.pi /180
    a = np.sin(Δφ/2) * np.sin(Δφ/2) +np.cos(φ1) * np.cos(φ2) *np.sin(Δλ/2) * np.sin(Δλ/2)
    c = 2*np.arctan2(np.sqrt(a), np.sqrt(1-a));
    return R * c
def gradient(height,distance):
    return np.arctan(height/distance) * 180/np.pi

for i in one_track.index:
    if (i == len(one_track)-1):
        break
    lat1= one_track.loc[i,'geometry'].y
    lat2= one_track.loc[i+1,'geometry'].y
    lon1= one_track.loc[i,'geometry'].x
    lon2= one_track.loc[i+1,'geometry'].x
    heightdiff = one_track.loc[i,'elevation']-one_track.loc[i+1,'elevation']
    one_track.loc[i+1,'seg_distance']= distance(lon1,lon2,lat1,lat2)
    one_track.loc[i,'gradient']= gradient(heightdiff,one_track.loc[i+1,'seg_distance'])

## Add interval time
j = 5
for i in one_track.index:
    one_track.loc[i, 'time_interval'] = j
    j = j+5

# Convert the speed unit to m/s
for i in one_track.index:
    one_track.loc[i, 'speed'] = one_track.loc[i, 'GPS Speed.value'] * 0.27777

## Getting the velocity equation & Calculate the acceleration

In [ ]:

## get the speed equation
time_interval = np.array(one_track['time_interval'])
speed = np.array(one_track['speed'])
idx = np.isfinite(time_interval) & np.isfinite(speed)

def get_equation(x,y):
    degree = 80
    coefs, res, _, _, _ = np.polyfit(x,y,degree, full = True)
    ffit = np.poly1d(coefs)
    #print (ffit)
    return ffit

speed_equation = get_equation(time_interval[idx], speed[idx])

## calculate the acceleration using the derivative
j = 0
for i in one_track.index:
    if one_track.loc[i, 'speed'] == 0:
        one_track.loc[i, 'Acceleration'] = 0
    else:
        one_track.loc[i, 'Acceleration'] = derivative(speed_equation, one_track.loc[i, 'time_interval'])
    j = j+5




## Filter the data

In [28]:
realistc_drive = one_track[['time','geometry','speed', 'track.length','seg_distance', 'sensor.fuelType', 'Acceleration', 'gradient']]
realistc_drive

,time,geometry,speed,track.length,seg_distance,sensor.fuelType,Acceleration,gradient
0,2020-03-12T17:43:10,POINT (7.65178 51.93495),0.821248,4.821775,NaN,diesel,3.948788,0.699533
1,2020-03-12T17:43:15,POINT (7.65183 51.93497),0.597299,4.821775,4.125327,diesel,0.001335,-0.277295
2,2020-03-12T17:43:20,POINT (7.65185 51.93480),4.027571,4.821775,19.109169,diesel,-0.658674,0.105049
3,2020-03-12T17:43:26,POINT (7.65236 51.93475),6.703896,4.821775,35.667954,diesel,-0.155244,-0.319264
4,2020-03-12T17:43:31,POINT (7.65274 51.93444),5.521024,4.821775,42.748974,diesel,0.457819,-0.242262
...,...,...,...,...,...,...,...,...
130,2020-03-12T17:54:08,POINT (7.65297 51.95383),0.000000,4.821775,2.003431,diesel,0.000000,NaN
131,2020-03-12T17:54:13,POINT (7.65299 51.95383),0.000000,4.821775,1.749858,diesel,0.000000,NaN
132,2020-03-12T17:54:18,POINT (7.65301 51.95381),0.001294,4.821775,1.993354,diesel,0.405947,NaN
133,2020-03-12T17:54:23,POINT (7.65301 51.95381),0.030658,4.821775,0.356133,diesel,-1.219897,NaN


## Define General Parameters for the cars

In [29]:
m = 1500      # mass of the car  "kg"
A = 2         # cross-sectional of the car "m²"
P_air = 1.2   # Air mass density "kg per m³" 
P_idle = 2    # Idle power "kW"
Cw = 0.3      # Air drag cofficient 
H_g = 8.8     # Calorific value gasoline "kWh/l"
H_d = 9.9     # Calorific value diesel "kWh/l"
g = 9.81      # Gravitational acceleration "m/s²"

##  Define Parameters for vehicle

In [30]:
# m = mass of the car  "kg" 
# A = cross-sectional of the car "m²"
# Cw = Air drag cofficient 
class Car:
    def __init__(self,m=1500,A=2,Cw=0.3):
        self.m = m
        self.A = A
        self.Cw = Cw

##  Define specific paramerer by class (Volkswagen)
volks = Car(1570,2.179,0.32)
##generalcar = car
car = Car()
# check mass
print(volks.m,car.m)

1570 1500


## Define Driving_resistence + engine_power

In [31]:
def engine_power(car,Cr,gradient,speed,acceleration):
    if speed > 0:
        power =speed*(0.5*car.Cw*car.A*P_air*pow(speed,2) #driving resistance
                      +car.m*g*Cr*np.cos(gradient) #rolling resistence
                      +car.m*g*np.sin(gradient) # climbing resistance
                      +car.m*+acceleration) # inertial resistance)
        return [power,power/speed]
    else:
        return [P_idle,0]

#engine_power(volks,0.02,0.01,0.821248,3.948788) 

## driving resistance

In [ ]:
## calculate the acceleration using the derivative # cr is set as an
for i in realistc_drive.index:
    ep = engine_power(car,0.02,realistc_drive.gradient[i],realistc_drive.speed[i],realistc_drive.Acceleration[i])
    realistc_drive.loc[i, 'engine_power'] = ep[0]
    realistc_drive.loc[i, 'driving_resistance'] = ep[1]

In [34]:
realistc_drive

,time,geometry,speed,track.length,seg_distance,sensor.fuelType,Acceleration,gradient,engine_power,driving_resistance
0,2020-03-12T17:43:10,POINT (7.65178 51.93495),0.821248,4.821775,NaN,diesel,3.948788,0.699533,12830.366109,15623.013565
1,2020-03-12T17:43:15,POINT (7.65183 51.93497),0.597299,4.821775,4.125327,diesel,0.001335,-0.277295,-2235.757535,-3743.113866
2,2020-03-12T17:43:20,POINT (7.65185 51.93480),4.027571,4.821775,19.109169,diesel,-0.658674,0.105049,3437.357430,853.456729
3,2020-03-12T17:43:26,POINT (7.65236 51.93475),6.703896,4.821775,35.667954,diesel,-0.155244,-0.319264,-30541.785808,-4555.826385
4,2020-03-12T17:43:31,POINT (7.65274 51.93444),5.521024,4.821775,42.748974,diesel,0.457819,-0.242262,-14060.430296,-2546.707089
...,...,...,...,...,...,...,...,...,...,...
130,2020-03-12T17:54:08,POINT (7.65297 51.95383),0.000000,4.821775,2.003431,diesel,0.000000,NaN,2.000000,0.000000
131,2020-03-12T17:54:13,POINT (7.65299 51.95383),0.000000,4.821775,1.749858,diesel,0.000000,NaN,2.000000,0.000000
132,2020-03-12T17:54:18,POINT (7.65301 51.95381),0.001294,4.821775,1.993354,diesel,0.405947,NaN,NaN,NaN
133,2020-03-12T17:54:23,POINT (7.65301 51.95381),0.030658,4.821775,0.356133,diesel,-1.219897,NaN,NaN,NaN


In [ ]:
#Retrieve summary statistics, driving_resistance and engine_power only calculated for 39 out 135 rows
realistc_drive.describe()

In [ ]:
plt.plot(realistc_drive['driving_resistance'])
plt.plot(realistc_drive['gradient'])
plt.show()

## Define Fuel Consumption for Volks

In [35]:
def q_volks(eng_pow, H_fuel, efc):
    cons = eng_pow / (H_fuel * efc)
    return cons

In [ ]:
#Calculates Engine Power for Volks
for i in realistc_drive.index:
    ep = engine_power(volks,0.02,realistc_drive.gradient[i],realistc_drive.speed[i],realistc_drive.Acceleration[i])
    realistc_drive.loc[i, 'eng_pw_volks'] = ep[0]

In [37]:
for i in realistc_drive.index:
    volks_cons = q_volks(realistc_drive.eng_pw_volks[i],H_d, 0.25) #Literature says eficiency is 0,25-0,35
    realistc_drive.loc[i, 'volks_q_diesel'] = volks_cons

realistc_drive

c:\users\pablo\anaconda3\envs\energydem\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\users\pablo\anaconda3\envs\energydem\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,time,geometry,speed,track.length,seg_distance,sensor.fuelType,Acceleration,gradient,engine_power,driving_resistance,eng_pw_volks,volks_q_diesel
0,2020-03-12T17:43:10,POINT (7.65178 51.93495),0.821248,4.821775,NaN,diesel,3.948788,0.699533,12830.366109,15623.013565,13429.139552,5425.914970
1,2020-03-12T17:43:15,POINT (7.65183 51.93497),0.597299,4.821775,4.125327,diesel,0.001335,-0.277295,-2235.757535,-3743.113866,-2340.084029,-945.488497
2,2020-03-12T17:43:20,POINT (7.65185 51.93480),4.027571,4.821775,19.109169,diesel,-0.658674,0.105049,3437.357430,853.456729,3600.483186,1454.740681
3,2020-03-12T17:43:26,POINT (7.65236 51.93475),6.703896,4.821775,35.667954,diesel,-0.155244,-0.319264,-30541.785808,-4555.826385,-31954.545207,-12910.927356
4,2020-03-12T17:43:31,POINT (7.65274 51.93444),5.521024,4.821775,42.748974,diesel,0.457819,-0.242262,-14060.430296,-2546.707089,-14709.588222,-5943.267969
...,...,...,...,...,...,...,...,...,...,...,...,...
130,2020-03-12T17:54:08,POINT (7.65297 51.95383),0.000000,4.821775,2.003431,diesel,0.000000,NaN,2.000000,0.000000,2.000000,0.808081
131,2020-03-12T17:54:13,POINT (7.65299 51.95383),0.000000,4.821775,1.749858,diesel,0.000000,NaN,2.000000,0.000000,2.000000,0.808081
132,2020-03-12T17:54:18,POINT (7.65301 51.95381),0.001294,4.821775,1.993354,diesel,0.405947,NaN,NaN,NaN,NaN,NaN
133,2020-03-12T17:54:23,POINT (7.65301 51.95381),0.030658,4.821775,0.356133,diesel,-1.219897,NaN,NaN,NaN,NaN,NaN


## Import OSM Network, in the same area of tracks

In [ ]:
import osmnx as ox

G = ox.graph_from_bbox(51.974045, 51.952343, 7.65083, 7.603312, network_type='drive')
G_projected = ox.project_graph(G)
ox.plot_graph(G_projected)

## Convert the OSM Netwok to geodataframe and filter the attributes

In [ ]:
nodes, streets = ox.graph_to_gdfs(G)
road_network = streets[['maxspeed','length','surface']]
road_network
#to check the type of surface "for rolling coff."
for i in road_network.index:
    if road_network.loc[i, 'surface'] == "asphalt":
        road_network.loc[i, 'rolling_resistance'] = 0.02 # source: engineeringtoolbox.com
    elif road_network.loc[i, 'surface'] == "cobblestone":
        road_network.loc[i, 'rolling_resistance'] = 0.015 # source: engineeringtoolbox.com
    elif road_network.loc[i, 'surface'] == "paving_stones":
        road_network.loc[i, 'rolling_resistance'] = 0.033 # source: The Automotive Chassis book
    else:
        road_network.loc[i, 'rolling_resistance'] = 0.02
        
road_network